In [1]:
from validation.pnp import camera_pos_from_output
from data.worldpos import pixel_to_world_pos_tensor

In [2]:
import PIL.Image
import numpy as np
import torch
from pathlib import Path

In [3]:
dimage = PIL.Image.open('chessd.png')
ndimage = np.array(dimage, dtype=np.uint8)


In [4]:
camera_to_world = np.loadtxt('chesst.txt')

In [5]:
world_tens = pixel_to_world_pos_tensor(torch.from_numpy(ndimage), camera_to_world, (585, 585), (320, 240))

In [6]:
world_tens.shape

torch.Size([120, 160, 3])

In [12]:
rm, tm = camera_pos_from_output(world_tens.numpy())

In [11]:
import cv2

In [9]:
cv2.__version__

'4.0.1'

In [10]:
world_pos = world_tens.numpy()

In [11]:
world_pos.reshape(-1, 3)

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.02130733,  0.12287685, -0.61505166],
       [ 0.02634493,  0.1519281 , -0.76046572],
       ...,
       [ 0.03721001,  0.19773327, -0.99227603],
       [ 0.03592406,  0.19605187, -0.99218169],
       [ 0.0359196 ,  0.19606102, -0.99219344]])

In [12]:
from data.worldpos import product2

h, w, *_channel = world_pos.shape
scale = 4
hr = range(0, h * scale, scale)
wr = range(0, w * scale, scale)
axis = product2(hr, wr)

In [14]:
axis.astype(np.float32)

array([[  0.,   0.],
       [  0.,   4.],
       [  0.,   8.],
       ...,
       [476., 628.],
       [476., 632.],
       [476., 636.]], dtype=float32)

In [11]:
import transforms3d.quaternions as tsq

In [16]:
cv2.Rodrigues(rm)

(array([[ 0.89986956, -0.12731672, -0.4171633 ],
        [-0.05376096,  0.91677665, -0.3957655 ],
        [ 0.43283314,  0.37856442,  0.81813474]]),
 array([[ 0.00692928, -0.20797953,  0.07842185, -0.21835957, -0.39404273,
         -0.88312387, -0.04152793,  0.88431423, -0.38721646],
        [ 0.43126374,  0.18732604,  0.87311461,  0.19872041, -0.00632205,
         -0.04163907, -0.87192425,  0.0783107 ,  0.42505476],
        [-0.03732008, -0.93726679,  0.20554668,  0.93845715, -0.03743122,
         -0.21418851,  0.19415231, -0.22456855,  0.00119553]]))

In [25]:
from validation.pnp import  combine_rotation_translation, homo_transform

In [9]:
combine_rotation_translation(rm, tm)

array([[ 0.89986956, -0.12731672, -0.4171633 , -0.60954383],
       [-0.05376096,  0.91677665, -0.3957655 , -0.78611842],
       [ 0.43283314,  0.37856442,  0.81813474,  0.19616935],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [20]:
from scipy.spatial.transform.rotation import Rotation

In [23]:
rot = Rotation.from_rotvec(rm.squeeze())

In [24]:
rot.as_matrix()

array([[ 0.89986956, -0.12731672, -0.4171633 ],
       [-0.05376096,  0.91677665, -0.3957655 ],
       [ 0.43283314,  0.37856442,  0.81813474]])

In [29]:
def homo_translation(translation: np.ndarray):
    id_mat = np.identity(4)
    id_mat.T[3, :-1] = translation
    return id_mat

In [13]:
def homo_transform(transform: np.ndarray):
    id_mat = np.identity(4)
    id_mat[:-1, :-1] = transform
    return id_mat#%%

est = homo_transform(cv2.Rodrigues(rm)[0])

In [16]:
def trans(p, mat):
    x, y, z = p
    nx, ny, nz, w = np.matmul([x, y, z, 1], mat)
    return [nx / w, ny / w, nz / w]

In [19]:
trans([1, 1, 1], camera_to_world)

[-0.7907246719684641, -0.6279393578669117, -0.977138492119077]

In [30]:
trans([1, 1, 1], inest)

[748.1809218369307, 683.2942508781705, 689.592411407295]

In [21]:
from validation.projection import default_intrinsics_matrix

In [22]:
import numpy.linalg as L

In [29]:
inest = est @ (homo_transform(default_intrinsics_matrix))

In [31]:
inest


array([[526.42369501, -74.48027857, 256.9850855 ,   0.        ],
       [-31.45016004, 536.31434208, 202.42712491,   0.        ],
       [253.20738686, 221.46018737, 230.180201  ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   1.        ]])